# 9章 制御の抽象化

## 新しい制御構造を作る例
似たようなことの繰り返しを、まるで言語の構文でサポートされているかのように見せる。

関数が第1級だからこそできることらしい。

In [5]:
// こんな風に、同じ処理を二回繰り返すコードがあるとする。
// これをもっとスマートに見せてコードを削減したい。

val a = 1
a + 1 + 1
a * 8 * 8

a: Int = 1
res4_1: Int = 3
res4_2: Int = 64

In [7]:
// 同じ関数を二度適用するメソッドを作る。

def twice(x: Double, op: Double => Double) = op(op(x))

defined function twice

In [8]:
// このメソッドに関数を渡すとあら不思議、
// まるで、新しい制御構造のようなものができる。

twice(5, _ + 1)

res7: Double = 7.0

In [9]:
twice(2, _ * 8)

res8: Double = 128.0

## 中括弧の使用
ある条件では、中括弧をかっこのかわりに使うことが許される。
これによって、より制御構文らしくなる。

In [10]:
// リソースの操作をするメソッドを定義してみる。
// (エラーが出ても必ずクローズされるようにするのが目的)
import java.io._
def withPrintWriter(file: File, op: PrintWriter => Unit) = {
    val writer = new PrintWriter(file)
    try {
        op(writer)
    } finally {
        writer.close()
    }
}

import java.io._

defined function withPrintWriter

In [11]:
// 以下のように使える（これではまだ制御構文っぽくない。
withPrintWriter(
    new File("tmp.txt"),
    writer => writer.println(new java.util.Date)
)

ちなみに、このような場合、関数にリスースを貸し出すので、ローンパターンと言うらしい。
上の例では、op に、PrintWriter を貸し出している。

In [12]:
// さらに、面白くするためにカリー化をする。
import java.io
def withPrintWriter(file: File)(op: PrintWriter => Unit) = {
    val writer = new PrintWriter(file)
    try {
        op(writer)
    } finally {
        writer.close()
    }
}

import java.io

defined function withPrintWriter

In [13]:
// 引数がひとつの場合のメソッド呼び出しには、
// 中括弧が使えるというシンタックス
import java.io
val file = new File("tmp.txt")                                                                                                                                                                         
withPrintWriter(file) { writer =>
   writer.println(new java.util.Date)
}

import java.io

file: File = tmp.txt

In [14]:
// そしてさらにこうすることができる！
import java.io
val file = new File("tmp.txt")                                                                                                                                                                         
withPrintWriter(file) {
   _.println(new java.util.Date)
}

import java.io

file: File = tmp.txt

ポイントは
* 引数がひとつだけの呼び出しは、中括弧を使うことができる。これで制御構文らしくなる。
* 引数が複数なければならない場合でも、カリー化によって呼び出しをチェーンに変更して、中括弧を使える。
* 関数値の引数が一回しか使われない場合、簡略化した記法をしようできる。

## 名前渡しパラメータの使用(by-name parameter)
名前渡しパラメータという機能を使うと、ある種のケースでより制御構文らしく書くことができる。

In [20]:
// 以下のように、Enable/Disable ができる Assertion を定義する。
var assertionEnabled = true
def myAssert(predicate: () => Boolean) =
    if (assertionEnabled && !predicate()) throw new AssertionError
    else println("Pass")

assertionEnabled: Boolean = false
defined function myAssert

In [22]:
// すると、このメソッドは以下のように使用できる。
myAssert(() => 5 > 3)
myAssert(() => 3 > 5)

Pass


: 

* しかし、この例では、`() =>`の部分がスマートでない。
* 空パラメータの指定を省略したい。
* こういった場合に、名前渡しパラメータを使うと、

In [23]:
def myAssert2(predicate: => Boolean) =
   if (assertionEnabled && !predicate) throw new AssertionError
    else println("Pass")

defined function myAssert2

* predicate の呼び出しに、()がないことに注意
* これで、空パラメータを省略できるようになった。

In [24]:
myAssert2(5 > 3)
myAssert2(5 > 5)

Pass


: 

 * この方式でいいのは、評価が遅延されることである。
 * 以下の例では、エラーを起こす0除算でもエラーにならない。predicate が評価されていないからである。

In [25]:
assertionEnabled = false
myAssert2(5 / 0 == 0)

Pass
